In [1]:
import base64
import datetime
import itertools
import json
import math
import operator
import os
import pickle
import random
import re
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
d = pd.concat([pd.read_csv('tmp/train_ids.csv', index_col='id'),
               pd.read_csv('tmp/test_ids.csv', index_col='id')])
print d.dtypes

interest_level    object
dtype: object


In [3]:
e = pd.read_csv('tmp/features_bathrooms.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

bathrooms               float64
num_bathrooms_bucket      int64
dtype: object


In [4]:
e = pd.read_csv('tmp/features_bedrooms.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

bedrooms               int64
num_bedrooms_bucket    int64
dtype: object


In [5]:
e = pd.read_csv('tmp/features_building_id.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

building_id                  object
building_id_was_missing     float64
num_apts_in_building          int64
num_apts_in_building_q10    float64
building_code               float64
dtype: object


In [6]:
e = pd.read_csv('tmp/features_created.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

created                           object
created_utc                       object
created_local_time                object
created_month                      int64
created_day_of_month               int64
created_dow                        int64
created_hour                       int64
created_minute                     int64
created_minute_of_the_day          int64
created_date                      object
created_hour_of_dataset            int64
created_distance_from_peak         int64
created_distance_from_peak_abs     int64
num_apts_same_hour                 int64
num_apts_prev_hour                 int64
num_apts_next_hour                 int64
num_apts_same_3hours               int64
num_apts_prev_3hours               int64
num_apts_next_3hours               int64
num_apts_same_9hours               int64
num_apts_prev_9hours               int64
num_apts_next_9hours               int64
num_apts_same_day                  int64
num_apts_prev_day                  int64
num_apts_next_da

In [7]:
e = pd.read_csv('tmp/features_description.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

count_non_alphanumeric_characters    float64
desc_num_bangs                         int64
desc_num_breaks                        int64
desc_num_emails                        int64
desc_num_nonalpha                    float64
desc_num_paragraphs                    int64
desc_num_parentheses                   int64
desc_num_phones                        int64
desc_num_website_redacted              int64
desc_num_words                         int64
num_characters                         int64
ratio_non_alphanumeric_characters    float64
desc_num_words_q100                  float64
desc_count_24                          int64
desc_count__blank                      int64
desc_count_access                      int64
desc_count_amazing                     int64
desc_count_amenities                   int64
desc_count_amp                         int64
desc_count_apartment                   int64
desc_count_apartments                  int64
desc_count_appliances                  int64
desc_count

In [8]:
e = pd.read_csv('tmp/features_display_address.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

daddr_num_words             int64
daddr_has_number            int64
daddr_has_ordinal           int64
daddr_num_upper_words       int64
daddr_count_10th            int64
daddr_count_11th            int64
daddr_count_13th            int64
daddr_count_14th            int64
daddr_count_1st             int64
daddr_count_25th            int64
daddr_count_29th            int64
daddr_count_2nd             int64
daddr_count_34th            int64
daddr_count_35th            int64
daddr_count_37th            int64
daddr_count_38th            int64
daddr_count_39th            int64
daddr_count_42nd            int64
daddr_count_46th            int64
daddr_count_47th            int64
daddr_count_54th            int64
daddr_count_57th            int64
daddr_count_5th             int64
daddr_count_63rd            int64
daddr_count_72nd            int64
daddr_count_79th            int64
daddr_count_82nd            int64
daddr_count_86th            int64
daddr_count_89th            int64
daddr_count_an

In [9]:
e = pd.read_csv('tmp/features_features.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

num_features                                              int64
top_10_feature_coverage                                 float64
top_30_feature_coverage                                 float64
top_10_feature_ratio                                    float64
top_30_feature_ratio                                    float64
features_count_24                                         int64
features_count__balconies                                 int64
features_count__chef                                      int64
features_count__dishwasher_                               int64
features_count__dryer                                     int64
features_count__eat                                       int64
features_count__elev                                      int64
features_count__elev_bldg_                                int64
features_count__exposed_brick_                            int64
features_count__gourmet_kitchen_                          int64
features_count__lndry_bldg_             

In [10]:
e = pd.read_csv('tmp/features_latlon.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

latitude                 float64
longitude                float64
cluster_id                 int64
cluster_size             float64
distance_cc              float64
price_q10_clust_all      float64
price_q100_clust_all     float64
price_q10_clust          float64
price_q100_clust         float64
area_density             float64
area_density_all         float64
area_density_time        float64
area_density_time_all    float64
area_distance_com        float64
price_area_q             float64
price_area_q_all         float64
price_area_time_q        float64
price_area_time_q_all    float64
dtype: object


In [11]:
e = pd.read_csv('tmp/features_manager_id.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

num_apts_at_manager           int64
num_apts_at_manager_q10     float64
num_apts_at_manager_q100    float64
private_manager             float64
manager_code                  int64
dtype: object


In [12]:
e = pd.read_csv('tmp/features_photos.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

photos              object
num_photos           int64
has_photos           int64
photos_per_room    float64
dtype: object


In [13]:
e = pd.read_csv('tmp/features_price.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

price                                int64
log_price                          float64
price_q10_all                      float64
price_q100_all                     float64
price_q10                          float64
price_q100                         float64
rounded_price                        int64
num_apts_with_approx_same_price      int64
dtype: object


In [14]:
e = pd.read_csv('tmp/features_street_address.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

saddr_num_words             int64
saddr_has_number            int64
saddr_has_ordinal           int64
saddr_num_upper_words       int64
daddr_saddr_same            int64
num_apts_with_same_saddr    int64
saddr_count_10              int64
saddr_count_100             int64
saddr_count_10th            int64
saddr_count_145             int64
saddr_count_14th            int64
saddr_count_150             int64
saddr_count_20              int64
saddr_count_200             int64
saddr_count_25              int64
saddr_count_2nd             int64
saddr_count_30              int64
saddr_count_300             int64
saddr_count_330             int64
saddr_count_34              int64
saddr_count_340             int64
saddr_count_34th            int64
saddr_count_350             int64
saddr_count_37              int64
saddr_count_37th            int64
saddr_count_39th            int64
saddr_count_42nd            int64
saddr_count_43              int64
saddr_count_45              int64
saddr_count_46

In [15]:
e = pd.read_csv('tmp/features_others.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

price_per_bedrooms                   float64
rooms_diff                           float64
rooms_sum                            float64
bathrooms_per_bedrooms               float64
hours_since_managers_last_listing    float64
dtype: object


In [16]:
d.interest_level.value_counts()

low       34284
medium    11229
high       3839
Name: interest_level, dtype: int64

In [17]:
for c in d.columns:
    if c != 'interest_level' and (d.dtypes[c] not in ('int64', 'float64')):
        del d[c]

In [18]:
tr = d[d.interest_level.notnull()]
print len(tr)

49352


In [19]:
te = d[d.interest_level.isnull()]
print len(te)

74659


In [20]:
print tr.values.shape
print te.values.shape

(49352, 610)
(74659, 610)


In [21]:
# target_num_map = {'high':0, 'medium':1, 'low':2}
# tr = np.array(trl.apply(lambda x: target_num_map[x]))

In [22]:
tr.to_csv('tmp/combined_train.csv')
te.to_csv('tmp/combined_test.csv')